In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [4]:
# constant of 1d tensor (vector)
a = tf.constant([2, 2], dtype=tf.int32, name="vector")
a.eval()

array([2, 2], dtype=int32)

In [5]:
# constant of 2x2 tensor (matrix)
b = tf.constant([[0, 1], [2, 3]], name="b")
b.eval()

array([[0, 1],
       [2, 3]], dtype=int32)

In [6]:
c = tf.zeros([2, 3], tf.int32) # [[0, 0, 0], [0, 0, 0]]
c.eval()

array([[0, 0, 0],
       [0, 0, 0]], dtype=int32)

In [7]:
d = tf.ones([2, 3], tf.int32) #  [[1, 1, 1], [1, 1, 1]]
d.eval()

array([[1, 1, 1],
       [1, 1, 1]], dtype=int32)

In [8]:
# create a tensor containing zeros, with shape and type as input_tensor
input_tensor = tf.constant([[1,1], [2,2], [3,3]], dtype=tf.float32)
e = tf.zeros_like(input_tensor)  #  [[0, 0], [0, 0], [0, 0]]
e.eval()

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]], dtype=float32)

In [9]:
f = tf.ones_like(input_tensor) # [[1, 1], [1, 1], [1, 1]]
f.eval()

array([[ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.]], dtype=float32)

In [10]:
#create variable a with scalar value
a = tf.Variable(2, name="scalar")
#create variable b as a vector
b = tf.Variable([2, 3], name="vector")
#create variable c as a 2x2 matrix
c = tf.Variable([[0, 1], [2, 3]], name="matrix")
# create variable W as 784 x 10 tensor, filled with zeros
W = tf.Variable(tf.zeros([784,10]))

In [11]:
# assign a * 2 to a and call that op a_times_two
a = tf.Variable(2, name="scalar")
a_times_two = a.assign(a*2) # an operation that assigns value a*2 to a

init = tf.global_variables_initializer() # an operation that initializes all variables
sess.run(init) # run the init operation with session
sess.run(a_times_two)

4

In [12]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
node3 = tf.add(node1, node2) 

print(node1) 
print(node2)
print(node3)

Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Add:0", shape=(), dtype=float32)


In [13]:
# create a directory to store our graph
import os

logs_dir = './graph'
if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)

In [14]:
sess = tf.Session()
print(sess.run([node1, node2]))
print(sess.run(node3))
sess.close() # close the session

[3.0, 4.0]
7.0


In [15]:
with tf.Session() as sess:
  # write operations to the event file
  writer = tf.summary.FileWriter(logs_dir, sess.graph) 
  print(sess.run([node1, node2]))
  print(sess.run(node3))
  # no need to write sess.close()
  
writer.close() 

[3.0, 4.0]
7.0


In [16]:
# create a placeholder of type float 32-bit, shape is a vector of 3 elements
a = tf.placeholder(tf.float32, shape=[3])
# create a constant of type float 32-bit, shape is a vector of 3 elements
b = tf.constant([5, 5, 5], tf.float32)
# use the placeholder as you would a constant or a variable
c = a + b # Short for tf.add(a, b)
with tf.Session() as sess:
# feed [1, 2, 3] to placeholder a via the dict {a: [1, 2, 3]}
# fetch value of c
    print(sess.run(c, feed_dict={a: [1, 2, 3]}))

[ 6.  7.  8.]


In [17]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])  # v.name == "foo/v:0"
    w = tf.get_variable("w", [1])  # w.name == "foo/w:0"
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v")  # The same as v above.

In [18]:
# clear used variables in jupyter notebook
%reset -fs 

In [21]:
"""The content of process_data.py"""

from collections import Counter
import random
import os
import sys
sys.path.append('..')
import zipfile

import numpy as np
from six.moves import urllib
import tensorflow as tf


# Parameters for downloading data
DOWNLOAD_URL = 'http://mattmahoney.net/dc/'
EXPECTED_BYTES = 31344016
DATA_FOLDER = 'data/'
FILE_NAME = 'text8.zip'


def make_dir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
    	pass

def download(file_name, expected_bytes):
    """ Download the dataset text8 if it's not already downloaded """
    file_path = DATA_FOLDER + file_name
    if os.path.exists(file_path):
        print("Dataset ready")
        return file_path
    file_name, _ = urllib.request.urlretrieve(DOWNLOAD_URL + file_name, file_path)
    file_stat = os.stat(file_path)
    if file_stat.st_size == expected_bytes:
        print('Successfully downloaded the file', file_name)
    else:
        raise Exception(
              'File ' + file_name +
              ' might be corrupted. You should try downloading it with a browser.')
    return file_path    
    
    
def read_data(file_path):
    """ Read data into a list of tokens"""
    with zipfile.ZipFile(file_path) as f:
        words = tf.compat.as_str(f.read(f.namelist()[0])).split()
        # tf.compat.as_str() converts the input into the string
    return words

def build_vocab(words, vocab_size):
    """ Build vocabulary of VOCAB_SIZE most frequent words """
    dictionary = dict()
    count = [('UNK', -1)]
    count.extend(Counter(words).most_common(vocab_size - 1))
    index = 0
    make_dir('processed')
    with open('processed/vocab_1000.tsv', "w") as f:
        for word, _ in count:
            dictionary[word] = index
            if index < 1000:
                f.write(word + "\n")
            index += 1
    index_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, index_dictionary

def convert_words_to_index(words, dictionary):
    """ Replace each word in the dataset with its index in the dictionary """
    return [dictionary[word] if word in dictionary else 0 for word in words]

def generate_sample(index_words, context_window_size):
    """ Form training pairs according to the skip-gram model. """
    for index, center in enumerate(index_words):
        context = random.randint(1, context_window_size)
        # get a random target before the center word
        for target in index_words[max(0, index - context): index]:
            yield center, target
        # get a random target after the center wrod
        for target in index_words[index + 1: index + context + 1]:
            yield center, target

def get_batch(iterator, batch_size):
    """ Group a numerical stream into batches and yield them as Numpy arrays. """
    while True:
        center_batch = np.zeros(batch_size, dtype=np.int32)
        target_batch = np.zeros([batch_size, 1])
        for index in range(batch_size):
            center_batch[index], target_batch[index] = next(iterator)
        yield center_batch, target_batch
        
def get_batch_gen(index_words, context_window_size, batch_size):
    """ Return a python generator that generates batches"""
    single_gen = generate_sample(index_words, context_window_size)
    batch_gen = get_batch(single_gen, batch_size)
    return batch_gen
        
def process_data(vocab_size):
    """ Read data, build vocabulary and dictionary"""
    file_path = download(FILE_NAME, EXPECTED_BYTES)
    words = read_data(file_path)
    dictionary, index_dictionary = build_vocab(words, vocab_size)
    index_words = convert_words_to_index(words, dictionary)
    del words # to save memory
    return index_words, dictionary, index_dictionary

In [23]:
vocab_size = 10000
window_sz = 5
batch_sz = 64
index_words, dictionary, index_dictionary = process_data(vocab_size)
batch_gen = get_batch_gen(index_words, window_sz, batch_sz)
X, y = next(batch_gen)

print(X.shape)
print(y.shape)

Successfully downloaded the file data/text8.zip
(64,)
(64, 1)


In [24]:
for i in range(10): # print out the pairs
  data = index_dictionary[X[i]]
  label = index_dictionary[y[i,0]]
  print('(', data, label,')')

( anarchism originated )
( originated anarchism )
( originated as )
( originated a )
( originated term )
( as anarchism )
( as originated )
( as a )
( as term )
( a anarchism )


In [25]:
for i in range(10): # print out the first 10 words in the text
  print(index_dictionary[index_words[i]], end=' ')

anarchism originated as a term of abuse first used against 

In [26]:
BATCH_SIZE = 128
dataset = tf.contrib.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.repeat()  # Repeat the input indefinitely.
dataset = dataset.batch(BATCH_SIZE) # stack BATCH_SIZE elements into one
iterator = dataset.make_one_shot_iterator() # iterator
next_batch = iterator.get_next() # an operation that gives the next batch

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.


In [27]:
with tf.Session() as sess:
  data, label = sess.run(next_batch)
  print(data.shape)
  print(label.shape)

(128,)
(128, 1)
